### 基本宣告

In [1]:
import pyspark

conf = pyspark.SparkConf().setMaster('local').setAppName('matrixMulti')
sc = pyspark.SparkContext(conf=conf)

21/10/04 11:39:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 用textFile將input讀進來後把每一行存成((M, row, col), val)的形式

In [2]:
def mapper1(x):
    l = x.split(',')
    res = []

    res.append(((l[0], int(l[1]), int(l[2])), int(l[3])))
    return res


distFile = sc.textFile("text.txt")
distFile = distFile.flatMap(mapper1)



### 將原始RDD根據key filter成m, n兩個新的RDD，同時先將(row, col)的排序存到k，再把m矩陣相同的row group起來；n矩陣相同的col

In [3]:
rdd_m = distFile.filter(lambda x: x[0][0] == 'M').map(
    lambda x: ((x[0][1], x[0][2]), x[1]))
rdd_n = distFile.filter(lambda x: x[0][0] == 'N').map(
    lambda x: ((x[0][1], x[0][2]), x[1]))
k = rdd_m.map(lambda x: x[0])

rdd_m = rdd_m.groupBy(lambda x: x[0][0]).map(
    lambda g: (tuple(x for x in g[1])))
rdd_n = rdd_n.groupBy(lambda x: x[0][1]).map(
    lambda g: (tuple(x for x in g[1])))



### 定義mapper2用來將算出來的積加總

In [4]:
def mapper2(x):
    res = 0

    for i in range(len(x[0])):
        res += x[0][i][1] * x[1][i][1]

    return res


## 主要邏輯與輸出
### 用cartesian去算出行列的乘積並且依照順序排序好，再用mapper2把行列的乘積加總起來
### 把上面先存起來的(row, col)的key跟算出來的值zip起來，最後再輸出至Outputfile

In [5]:
matrix = rdd_m.cartesian(rdd_n).sortBy(lambda x: x[0][0][0]).map(mapper2)
k = k.zip(matrix)
k = k.map(lambda x: (x[0][0], x[0][1], x[1]))

res = k.collect()
with open('Outputfile.txt', 'w+') as f:
    for i in res:
        f.write(f'{i[0]},{i[1]},{i[2]}\n')

sc.stop()
